In [ ]:
import pandas as pd

df = pd.read_csv('../data/协作表 - V1 - 待审查.csv')
df = df[['自我肯定语-V1', '字数']]

Index(['序号', '自我肯定语-V1', '字数', '负责人', '标注', '备注', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21'],
      dtype='object')

In [7]:

df.head()

,自我肯定语-V1,字数
0,我的价值绝大部分由我自身决定。,15
1,我对自己的信任是成功的关键。,14
2,我拥有超越困难的内在力量。,13
3,我接受自己的不完美，依然自信满满。,17
4,我的每一步都是自我肯定的证明。,15


In [10]:
import sys
sys.path.append('../src')
from kimi_api import client

In [ ]:
role_prompt = """
你是一个情感分析专家，负责对句子进行四个维度的标注：
1. 合集（句子一对一）
    选项：
        情感疗愈: 提及和“情感”、“亲密关系”相关的词语
        自信: 有力量感，自信感，提及“我能”或其他自我认知矫正的词语
        平和心境: 提及“情绪”或具体的情绪感受，提及“平和”、“安宁”、“感恩已拥有的”
        治愈之旅: 有非常明显的安慰性，提及自我接纳
        人际交往: 提及和他人、沟通相关的词语
        自我关怀: 提及爱自己、健康、快乐、休息、身体、运动、饮食、身体动作如呼吸等相关的词语
        个人成长: 提及和成长、事业、工作、学习、产出、创造相关的词语
        成为英雄: 提及和勇敢勇气、原谅、牺牲、遗产、理想相关的词语
2. 感情状况（句子一对多）
3. 最近的感觉（句子一对多）
4. 什么让你有这种感觉（句子一对多）

请根据提供的句子特征进行标注，并返回JSON格式的结果。以下是返回结果的JSON示例：

{
  "合集": "情感疗愈",
  "感情状况": ["正在恋爱", "快乐地单身着"],
  "最近的感觉": ["开心", "很好"],
  "什么让你有这种感觉": ["家庭", "朋友"]
}

请确保返回的JSON格式与示例一致，并根据句子的实际内容进行标注。
"""

In [ ]:
import json
def get_annotations(sentence):
    message = f"请对以下句子进行标注：{sentence}"
    messages = [
        {"role": "system", "content": role_prompt},
        {"role": "user", "content": message}
    ]
    completion = client.chat.completions.create(
        model="moonshot-v1-auto",
        messages=messages,
        temperature=1,
        response_format={"type": "json_object"},  # 确保返回 JSON 格式
        n=1  # 请求返回1个结果
    )
    response = completion.choices[0].message.content.strip()
    try:
        response_dict = json.loads(response)
        return response_dict
    except json.JSONDecodeError:
        return {"error": "Invalid JSON response"}

In [ ]:
results = []
for index, row in df.iterrows():
    sentence = row['自我肯定语-V1']
    annotations = get_annotations(sentence)
    results.append({
        "句子": sentence,
        "合集": annotations.get("合集", ""),
        "感情状况": annotations.get("感情状况", ""),
        "最近的感觉": annotations.get("最近的感觉", ""),
        "什么让你有这种感觉": annotations.get("什么让你有这种感觉", "")
    })

# 将结果转换为DataFrame并导出为CSV
results_df = pd.DataFrame(results)
results_df.to_csv('../data/标注结果.csv', index=False)

print("标注完成，结果已导出为标注结果.csv")